In [63]:
# imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import pandas_profiling

In [84]:
def generate_time_series(start_date_str, end_date_str, interval_min):
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d %H:%M")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d %H:%M")
    interval = timedelta(hours=interval_min/60)
    output = []
    while start_date <= end_date:
        output.append(start_date.strftime("%Y-%m-%d %H:%M:%S"))
        start_date += interval
    return output

In [3]:
# navigate to price files
os.chdir("C:\\Users\\joche\\FIM Kernkompetenzzentrum\\Projekt VIdES - Dokumente\\General\\07_Arbeitsordner\\02_Daten_und_Simulationsvorbereitung\\Strompreise\\historische strompreise")

In [5]:
# read in and display historic price data

# create dummy series and df to append to
series = pd.Series(dtype="float64")
df = pd.read_csv(os.listdir()[0])
for i in os.listdir():
    # extract column name from file name
    s = "price_"
    s += i.split(".",2)[0][-4:]
    # read in file
    tmp = pd.read_csv(i, header=None)
    # assign to series
    series = pd.concat([series, tmp.iloc[:,0]])
    # assign to df
    tmp.set_axis([s], axis=1, inplace=True)
    df[s] = tmp
# disregard first column of df since it's a duplicate
df_price = df.iloc[:,1:]
# truncate at index 8760
df_price = df_price.truncate(after=8760)
# add mean column
df_price['price_2016-2020_mean'] = df_price.iloc[:,:5].mean(axis=1)
# interpolate missing values (at the end)
df_price.interpolate()

# reset series index, transform to df, and rename column
df_price_series = pd.DataFrame(series.reset_index()[0])
df_price_series.columns = ["price"]
#df_price_series = df_price_series.columns = ["price"]
# insert two empty rows and interpolate missing values
df_price_series.loc[df_price_series.shape[0]] = [np.nan]
df_price_series.loc[df_price_series.shape[0]] = [np.nan]
df_price_series.interpolate(inplace=True)
# generate hourly time series from 2016 to 2020
time_series = generate_time_series(start_date_str="2016-01-01 00:00", end_date_str="2020-12-31 23:00", interval_min=60)
# transform time series to df
df_time_series = pd.DataFrame(time_series, index=time_series)
# assign index of df_time_series to price_series
df_price_series.index = df_time_series.index

In [ ]:
# summary statistics of df_price
price_report = pandas_profiling.ProfileReport(df_price.iloc[:,:-1])
price_report.to_file("price_report.html")

In [31]:
# navigate to pv and wind load profile file
os.chdir('C:\\Users\\joche\\FIM Kernkompetenzzentrum\\Projekt VIdES - Dokumente\\General\\07_Arbeitsordner\\02_Daten_und_Simulationsvorbereitung\\Strompreise')
file1 = "2022 06 07 Strompreise.xlsx"
file2 = "pvLoad_windLoad_electricity_timeseries.xlsx"
df_pv = pd.read_excel(file1, sheet_name="pv_load_profiles")
df_wind = pd.read_excel(file1, sheet_name="wind_load_profiles")
df_pv_series = pd.read_excel(file2, sheet_name="pv")
df_wind_series = pd.read_excel(file2, sheet_name="wind")

# df_pv and df_wind: Convert date to datetime and set as index
df_pv['datetime'] = pd.to_datetime(df_pv['datetime'])
df_wind['datetime'] = pd.to_datetime(df_wind['datetime'])
df_pv.set_index('datetime', inplace=True)
df_wind.set_index('datetime', inplace=True)
# df_pv_series and df_wind_series: Convert date to datetime, set as index and remove cap_factor
df_pv_series['datetime'] = pd.to_datetime(df_pv_series['datetime'])
df_wind_series['datetime'] = pd.to_datetime(df_wind_series['datetime'])
df_pv_series.set_index('datetime', inplace=True)
df_wind_series.set_index('datetime', inplace=True)
df_pv_series.drop('pv_cap_factor', axis=1, inplace=True)
df_wind_series.drop('wind_cap_factor', axis=1, inplace=True)

In [ ]:
# summary statistics of df_pv and df_wind
df_pv_report = pandas_profiling.ProfileReport(df_pv.iloc[:,1:])
df_pv_report.to_file("pv_report.html")
df_wind_report = pandas_profiling.ProfileReport(df_wind.iloc[:,1:])
df_wind_report.to_file("wind_report.html")

In [72]:
# more summary statistics of df_pv and df_wind
display(df_pv.iloc[:,1:].describe().transpose())
display(df_wind.iloc[:,1:].describe().transpose())

,count,mean,std,min,25%,50%,75%,max
pv_2016_mw,8760.0,5735.128135,8378.983631,0.0,0.0,90.566,10007.543,32920.741
pv_2017_mw,8760.0,6354.577193,9283.994539,0.0,0.0,100.348,11088.454,36476.498
pv_2018_mw,8760.0,6627.383015,9682.562009,0.0,0.0,104.656,11564.488,38042.456
pv_2019_mw,8760.0,6736.935956,9842.618119,0.0,0.0,106.386,11755.653,38671.311
pv_2020_mw,8760.0,6892.336486,10069.657249,0.0,0.0,108.840,12026.820,39563.340
pv_2030_mw,8760.0,16010.434116,23391.136558,0.0,0.0,252.828,27937.494,91902.978


,count,mean,std,min,25%,50%,75%,max
wind_2016_mw,8760.0,9067.160101,7518.767627,12.2037,3652.9742,6771.01955,12118.2741,38734.5438
wind_2017_mw,8760.0,9426.913202,7817.085947,12.6879,3797.9114,7039.66985,12599.0847,40271.3946
wind_2018_mw,8760.0,10065.508391,8346.628690,13.5474,4055.1884,7516.54910,13452.5682,42999.4476
wind_2019_mw,8760.0,10932.348422,9065.439067,14.7141,4404.4206,8163.87315,14611.1013,46702.5534
wind_2020_mw,8760.0,12002.469016,9952.815929,16.1544,4835.5504,8962.99960,16041.3192,51274.0656
wind_2030_mw,8760.0,44579.070776,36966.334605,60.0000,17960.0000,33290.00000,59580.0000,190440.0000


In [ ]:
# plot pv and wind load per year and month
pv_overview = df_pv_series.groupby([df_pv_series.index.year,df_pv_series.index.month]).sum()
ax = pv_overview.unstack(level=0).plot(kind='bar', subplots=True, figsize=(10,8))
plt.savefig('pv_overview.png')
wind_overview = df_wind_series.groupby([df_wind_series.index.year,df_wind_series.index.month]).sum()
ax = wind_overview.unstack(level=0).plot(kind='bar', subplots=True, figsize=(10,8))
plt.savefig('wind_overview.png')

In [73]:
df_price

,price_2016,price_2017,price_2018,price_2019,price_2020,price_2016-2020_mean
0,22.39,20.96,-5.27,28.32,36.550,20.590
1,20.59,20.90,-29.99,10.07,32.320,10.778
2,16.81,18.13,-56.65,-4.08,30.850,1.012
3,17.41,16.03,-63.14,-9.91,30.140,-1.894
4,17.02,16.43,-64.62,-7.41,30.170,-1.682
...,...,...,...,...,...,...
8756,41.03,4.06,50.01,42.20,43.220,36.104
8757,40.55,5.30,45.06,39.74,37.430,33.616
8758,33.06,1.86,48.93,38.88,38.540,32.254
8759,36.10,-0.92,30.31,37.39,35.335,27.643


In [81]:
os.getcwd()

'C:\\Users\\joche\\FIM Kernkompetenzzentrum\\Projekt VIdES - Dokumente\\General\\07_Arbeitsordner\\02_Daten_und_Simulationsvorbereitung\\Strompreise'